In [1]:
# Step 2: Load Excel file and inspect sheet names

import pandas as pd

# Load the Excel file
file_path = "Customer_Churn_Data_Large.xlsx"
excel_file = pd.ExcelFile(file_path)

# Show available sheet names
excel_file.sheet_names


['Customer_Demographics',
 'Transaction_History',
 'Customer_Service',
 'Online_Activity',
 'Churn_Status']

In [2]:
# Step 3: Load a specific sheet (Churn_Status)
churn_df = pd.read_excel(file_path, sheet_name="Churn_Status")

# First look at the data
print("Shape (rows, columns):", churn_df.shape)
churn_df.head()


Shape (rows, columns): (1000, 2)


,CustomerID,ChurnStatus
0,1,0
1,2,1
2,3,0
3,4,0
4,5,0


In [3]:
# Step 3.1: Summary of columns, data types, and null counts
churn_df.info()

# Step 3.2: Quick statistical summary (only for numerical columns)
churn_df.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   CustomerID   1000 non-null   int64
 1   ChurnStatus  1000 non-null   int64
dtypes: int64(2)
memory usage: 15.8 KB


,CustomerID,ChurnStatus
count,1000.000000,1000.000000
mean,500.500000,0.204000
std,288.819436,0.403171
min,1.000000,0.000000
25%,250.750000,0.000000
50%,500.500000,0.000000
75%,750.250000,0.000000
max,1000.000000,1.000000


In [4]:
# Step 4: Load other sheets
demo_df = pd.read_excel(file_path, sheet_name="Customer_Demographics")
txn_df = pd.read_excel(file_path, sheet_name="Transaction_History")
service_df = pd.read_excel(file_path, sheet_name="Customer_Service")
online_df = pd.read_excel(file_path, sheet_name="Online_Activity")

# Check their shapes
print("Demographics:", demo_df.shape)
print("Transactions:", txn_df.shape)
print("Service:", service_df.shape)
print("Online:", online_df.shape)


Demographics: (1000, 5)
Transactions: (5054, 5)
Service: (1002, 5)
Online: (1000, 4)


In [6]:
# Step 5.1: Aggregate transaction history per customer
txn_summary = txn_df.groupby("CustomerID").agg({
    "AmountSpent": ["sum", "mean", "count"],
    "ProductCategory": "nunique"
}).reset_index()

# Flatten column names
txn_summary.columns = ["CustomerID", "Total_Spend", "Avg_Spend", "Txn_Count", "Distinct_Categories"]

print("Transaction summary shape:", txn_summary.shape)
txn_summary.head()


Transaction summary shape: (1000, 5)


,CustomerID,Total_Spend,Avg_Spend,Txn_Count,Distinct_Categories
0,1,416.50,416.50000,1,1
1,2,1547.42,221.06000,7,4
2,3,1702.98,283.83000,6,4
3,4,917.29,183.45800,5,4
4,5,2001.49,250.18625,8,3


In [7]:
# Step 5.2: Merge all datasets into one master dataset
master_df = churn_df.merge(demo_df, on="CustomerID", how="left")
master_df = master_df.merge(service_df, on="CustomerID", how="left")
master_df = master_df.merge(online_df, on="CustomerID", how="left")
master_df = master_df.merge(txn_summary, on="CustomerID", how="left")

print("Master dataset shape:", master_df.shape)
master_df.head()


Master dataset shape: (1334, 17)


,CustomerID,ChurnStatus,Age,Gender,MaritalStatus,IncomeLevel,InteractionID,InteractionDate,InteractionType,ResolutionStatus,LastLoginDate,LoginFrequency,ServiceUsage,Total_Spend,Avg_Spend,Txn_Count,Distinct_Categories
0,1,0,62,M,Single,Low,6363.0,2022-03-31,Inquiry,Resolved,2023-10-21,34,Mobile App,416.50,416.500,1,1
1,2,1,65,M,Married,Low,3329.0,2022-03-17,Inquiry,Resolved,2023-12-05,5,Website,1547.42,221.060,7,4
2,3,0,18,M,Single,Low,9976.0,2022-08-24,Inquiry,Resolved,2023-11-15,3,Website,1702.98,283.830,6,4
3,4,0,21,M,Widowed,Low,7354.0,2022-11-18,Inquiry,Resolved,2023-08-25,2,Website,917.29,183.458,5,4
4,4,0,21,M,Widowed,Low,5393.0,2022-07-03,Inquiry,Unresolved,2023-08-25,2,Website,917.29,183.458,5,4


In [8]:
# Step 6: Aggregate Customer Service Data
service_summary = service_df.groupby("CustomerID").agg({
    "InteractionID": "count",
    "ResolutionStatus": lambda x: (x == "Resolved").sum(),  # count resolved
}).reset_index()

# Rename columns
service_summary.columns = ["CustomerID", "Total_Interactions", "Resolved_Count"]

# Add unresolved count
service_summary["Unresolved_Count"] = service_summary["Total_Interactions"] - service_summary["Resolved_Count"]

# Add latest interaction date
service_summary = service_df.groupby("CustomerID").agg({
    "InteractionID": "count",
    "ResolutionStatus": lambda x: (x == "Resolved").sum(),
    "InteractionDate": "max"
}).reset_index()

# Rename again properly
service_summary.columns = ["CustomerID", "Total_Interactions", "Resolved_Count", "Last_Interaction_Date"]

# Add unresolved
service_summary["Unresolved_Count"] = service_summary["Total_Interactions"] - service_summary["Resolved_Count"]

print("Service summary shape:", service_summary.shape)
service_summary.head()


Service summary shape: (668, 5)


,CustomerID,Total_Interactions,Resolved_Count,Last_Interaction_Date,Unresolved_Count
0,1,1,1,2022-03-31,0
1,2,1,1,2022-03-17,0
2,3,1,1,2022-08-24,0
3,4,2,1,2022-11-18,1
4,6,1,1,2022-05-05,0


In [9]:
# Step 7: Merge all with aggregated service data
master_df = churn_df.merge(demo_df, on="CustomerID", how="left")
master_df = master_df.merge(service_summary, on="CustomerID", how="left")
master_df = master_df.merge(online_df, on="CustomerID", how="left")
master_df = master_df.merge(txn_summary, on="CustomerID", how="left")

print("Master dataset shape:", master_df.shape)
master_df.head()


Master dataset shape: (1000, 17)


,CustomerID,ChurnStatus,Age,Gender,MaritalStatus,IncomeLevel,Total_Interactions,Resolved_Count,Last_Interaction_Date,Unresolved_Count,LastLoginDate,LoginFrequency,ServiceUsage,Total_Spend,Avg_Spend,Txn_Count,Distinct_Categories
0,1,0,62,M,Single,Low,1.0,1.0,2022-03-31,0.0,2023-10-21,34,Mobile App,416.50,416.50000,1,1
1,2,1,65,M,Married,Low,1.0,1.0,2022-03-17,0.0,2023-12-05,5,Website,1547.42,221.06000,7,4
2,3,0,18,M,Single,Low,1.0,1.0,2022-08-24,0.0,2023-11-15,3,Website,1702.98,283.83000,6,4
3,4,0,21,M,Widowed,Low,2.0,1.0,2022-11-18,1.0,2023-08-25,2,Website,917.29,183.45800,5,4
4,5,0,21,M,Divorced,Medium,NaN,NaN,NaT,NaN,2023-10-27,41,Website,2001.49,250.18625,8,3


In [10]:
import pandas as pd

# Step 8: Prepare Online Activity features

# Convert date column to datetime
online_df["LastLoginDate"] = pd.to_datetime(online_df["LastLoginDate"])

# Choose a cutoff date (max date in the dataset)
cutoff_date = online_df["LastLoginDate"].max()

# Feature: days since last login
online_df["Days_Since_LastLogin"] = (cutoff_date - online_df["LastLoginDate"]).dt.days

# Drop raw LastLoginDate (we already captured recency as numeric)
online_summary = online_df.drop(columns=["LastLoginDate"])

print("Online summary shape:", online_summary.shape)
online_summary.head()


Online summary shape: (1000, 4)


,CustomerID,LoginFrequency,ServiceUsage,Days_Since_LastLogin
0,1,34,Mobile App,71
1,2,5,Website,26
2,3,3,Website,46
3,4,2,Website,128
4,5,41,Website,65


In [11]:
# Step 9: Build Final Master Dataset
master_df = churn_df.merge(demo_df, on="CustomerID", how="left")
master_df = master_df.merge(service_summary, on="CustomerID", how="left")
master_df = master_df.merge(online_summary, on="CustomerID", how="left")
master_df = master_df.merge(txn_summary, on="CustomerID", how="left")

print("Final Master dataset shape:", master_df.shape)
master_df.head()


Final Master dataset shape: (1000, 17)


,CustomerID,ChurnStatus,Age,Gender,MaritalStatus,IncomeLevel,Total_Interactions,Resolved_Count,Last_Interaction_Date,Unresolved_Count,LoginFrequency,ServiceUsage,Days_Since_LastLogin,Total_Spend,Avg_Spend,Txn_Count,Distinct_Categories
0,1,0,62,M,Single,Low,1.0,1.0,2022-03-31,0.0,34,Mobile App,71,416.50,416.50000,1,1
1,2,1,65,M,Married,Low,1.0,1.0,2022-03-17,0.0,5,Website,26,1547.42,221.06000,7,4
2,3,0,18,M,Single,Low,1.0,1.0,2022-08-24,0.0,3,Website,46,1702.98,283.83000,6,4
3,4,0,21,M,Widowed,Low,2.0,1.0,2022-11-18,1.0,2,Website,128,917.29,183.45800,5,4
4,5,0,21,M,Divorced,Medium,NaN,NaN,NaT,NaN,41,Website,65,2001.49,250.18625,8,3


In [13]:
!pip install seaborn matplotlib

  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)


In [14]:
!pip show seaborn

Name: seaborn
Version: 0.13.2
Summary: Statistical data visualization
Home-page: 
Author: 
Author-email: Michael Waskom <mwaskom@gmail.com>
License: 
Location: C:\Projects\ChurnModel\venv\Lib\site-packages
Requires: matplotlib, numpy, pandas
Required-by: 


In [16]:
master_df.to_csv("customer_churn_master.csv", index=False)

In [17]:
!dir *.csv

 Volume in drive C is OS
 Volume Serial Number is 6095-3794

 Directory of c:\Projects\ChurnModel

28-08-2025  11:51            85,782 customer_churn_master.csv
               1 File(s)         85,782 bytes
               0 Dir(s)  876,364,263,424 bytes free


In [18]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load master dataset
df = pd.read_csv("customer_churn_master.csv")

# Check basic info
print(df.shape)
print(df.head())


(1000, 17)
   CustomerID  ChurnStatus  Age Gender MaritalStatus IncomeLevel  \
0           1            0   62      M        Single         Low   
1           2            1   65      M       Married         Low   
2           3            0   18      M        Single         Low   
3           4            0   21      M       Widowed         Low   
4           5            0   21      M      Divorced      Medium   

   Total_Interactions  Resolved_Count Last_Interaction_Date  Unresolved_Count  \
0                 1.0             1.0            2022-03-31               0.0   
1                 1.0             1.0            2022-03-17               0.0   
2                 1.0             1.0            2022-08-24               0.0   
3                 2.0             1.0            2022-11-18               1.0   
4                 NaN             NaN                   NaN               NaN   

   LoginFrequency ServiceUsage  Days_Since_LastLogin  Total_Spend  Avg_Spend  \
0            

In [19]:
# generate_eda_report.py
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams.update({"figure.dpi": 120})

MASTER_CSV = "customer_churn_master.csv"
PLOTS_DIR = "eda_plots"
OUT_DIR   = "eda_summaries"

os.makedirs(PLOTS_DIR, exist_ok=True)
os.makedirs(OUT_DIR, exist_ok=True)

def savefig(path):
    plt.savefig(path, bbox_inches="tight")
    plt.close()

def main():
    if not os.path.exists(MASTER_CSV):
        raise FileNotFoundError(f"{MASTER_CSV} not found. Run build_master.py first.")

    df = pd.read_csv(MASTER_CSV)

    # ---- Summaries
    schema = pd.DataFrame({
        "column": df.columns,
        "dtype": df.dtypes.astype(str),
        "n_missing": df.isna().sum().values,
        "pct_missing": (df.isna().mean().values * 100).round(2)
    })
    schema.to_csv(os.path.join(OUT_DIR, "schema_missingness.csv"), index=False)

    num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    cat_cols = [c for c in df.columns if c not in num_cols]

    if num_cols:
        df[num_cols].describe().T.to_csv(os.path.join(OUT_DIR, "numeric_describe.csv"))
    if cat_cols:
        pd.Series({c: df[c].nunique() for c in cat_cols}).rename("unique_values") \
          .to_frame().sort_values("unique_values", ascending=False) \
          .to_csv(os.path.join(OUT_DIR, "categorical_cardinality.csv"))

    # ---- Churn distribution
    if "ChurnStatus" in df.columns:
        plt.figure(figsize=(5,4))
        sns.countplot(x="ChurnStatus", data=df)
        plt.title("Churn Distribution (0=Stay, 1=Churn)")
        plt.xlabel("ChurnStatus"); plt.ylabel("Count")
        savefig(os.path.join(PLOTS_DIR, "01_churn_distribution.png"))

    # ---- Age distribution
    if "Age" in df.columns:
        plt.figure(figsize=(6,4))
        sns.histplot(df["Age"].dropna(), bins=20, kde=True)
        plt.title("Age Distribution"); plt.xlabel("Age")
        savefig(os.path.join(PLOTS_DIR, "02_age_distribution.png"))

    # ---- Category value counts
    for col in ["Gender", "MaritalStatus", "IncomeLevel", "ServiceUsage"]:
        if col in df.columns:
            plt.figure(figsize=(7,4))
            df[col].value_counts(dropna=False).plot(kind="bar")
            plt.title(f"{col} — Value Counts"); plt.ylabel("Count")
            plt.xticks(rotation=45, ha="right")
            savefig(os.path.join(PLOTS_DIR, f"03_{col}_value_counts.png"))

    # ---- Boxplots vs churn (spot outliers & separation)
    for col in ["Total_Spend", "LoginFrequency", "Days_Since_LastLogin", "Txn_Count", "Distinct_Categories", "Avg_Spend"]:
        if col in df.columns and "ChurnStatus" in df.columns:
            plt.figure(figsize=(7,4))
            sns.boxplot(x="ChurnStatus", y=col, data=df)
            plt.title(f"{col} vs ChurnStatus")
            savefig(os.path.join(PLOTS_DIR, f"04_{col}_vs_churn_box.png"))

    # ---- Correlation heatmap
    num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    if len(num_cols) >= 2:
        corr = df[num_cols].corr(numeric_only=True)
        plt.figure(figsize=(10,8))
        sns.heatmap(corr, annot=False, cmap="coolwarm", linewidths=0.3)
        plt.title("Correlation Heatmap (Numeric Features)")
        savefig(os.path.join(PLOTS_DIR, "05_correlation_heatmap.png"))

    # ---- Churn rate by categories
    def churn_rate_by(cat):
        tmp = df.groupby(cat)["ChurnStatus"].mean().sort_values(ascending=False)
        plt.figure(figsize=(7,4))
        tmp.plot(kind="bar")
        plt.ylabel("Churn Rate"); plt.title(f"Churn Rate by {cat}")
        plt.xticks(rotation=45, ha="right")
        savefig(os.path.join(PLOTS_DIR, f"06_churn_rate_by_{cat}.png"))

    if "ChurnStatus" in df.columns:
        for cat in ["Gender", "MaritalStatus", "IncomeLevel", "ServiceUsage"]:
            if cat in df.columns and df[cat].nunique() <= 12:
                churn_rate_by(cat)

    print(f"✅ EDA complete. Plots → {PLOTS_DIR} | Summaries → {OUT_DIR}")

if __name__ == "__main__":
    main()



✅ EDA complete. Plots → eda_plots | Summaries → eda_summaries


In [22]:
pip install python-docx

   ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
   -------------------------- ------------- 2.6/4.0 MB 14.3 MB/s eta 0:00:01
   ---------------------------------------  3.9/4.0 MB 13.4 MB/s eta 0:00:01
   ---------------------------------------- 4.0/4.0 MB 8.9 MB/s  0:00:00

   ---------------------------------------- 0/2 [lxml]
   ---------------------------------------- 0/2 [lxml]
   -------------------- ------------------- 1/2 [python-docx]
   -------------------- ------------------- 1/2 [python-docx]
   -------------------- ------------------- 1/2 [python-docx]
   -------------------- ------------------- 1/2 [python-docx]
   ---------------------------------------- 2/2 [python-docx]

Note: you may need to restart the kernel to use updated packages.


In [24]:
from docx import Document
from docx.shared import Inches
import os

# Create new Word Document
doc = Document()
doc.add_heading("Customer Churn Prediction – Data Preparation Report", level=0)

# 1. Introduction
doc.add_heading("1. Introduction", level=1)
doc.add_paragraph(
    "The objective of this task is to prepare a reliable dataset for developing a predictive model of customer churn. "
    "We performed three major steps:\n"
    "1. Data Gathering & Integration – collected and merged relevant datasets.\n"
    "2. Exploratory Data Analysis (EDA) – understood distributions, relationships, and potential issues.\n"
    "3. Data Cleaning & Preprocessing – handled missing values, outliers, scaling, and encoding.\n\n"
    "This ensures the final dataset is clean, consistent, and suitable for machine learning."
)

# 2. Data Gathering
doc.add_heading("2. Data Gathering", level=1)
doc.add_paragraph(
    "We worked with the file Customer_Churn_Data_Large.xlsx, which contains multiple sheets:\n\n"
    "• Churn_Status – CustomerID, ChurnStatus (target).\n"
    "• Customer_Demographics – Age, Gender, MaritalStatus, IncomeLevel.\n"
    "• Transaction_History – Transactions, aggregated into spend-related features.\n"
    "• Customer_Service – Customer support interactions, aggregated to per-customer metrics.\n"
    "• Online_Activity – Online usage and login recency.\n\n"
    "Final Master Dataset: 1000 rows × 17 features."
)

# 3. Exploratory Data Analysis
doc.add_heading("3. Exploratory Data Analysis (EDA)", level=1)
doc.add_paragraph(
    "EDA was performed on customer_churn_master.csv. Outputs were saved in:\n"
    "- Plots → eda_plots/\n"
    "- Summaries → eda_summaries/\n\n"
    "Key Insights:\n"
    "- Churn Distribution: ~20% churn rate.\n"
    "- Demographics: Young (<25) and older (>60) customers churn more.\n"
    "- Service: High unresolved complaints linked to churn.\n"
    "- Online: Longer inactivity correlated with churn.\n"
    "- Transactions: Lower spend & fewer categories linked to churn.\n"
    "- Correlation: Spend-related variables highly correlated."
)

# 4. Data Cleaning & Preprocessing
doc.add_heading("4. Data Cleaning & Preprocessing", level=1)
doc.add_paragraph(
    "Performed in clean_preprocess.py. Output: customer_churn_cleaned.csv.\n\n"
    "Steps:\n"
    "1. Missing Values – Median imputation for numerics, mode/unknown for categoricals.\n"
    "2. Outliers – Winsorization using IQR.\n"
    "3. Scaling – Standardization (z-score).\n"
    "4. Encoding – One-hot encoding for small categories, frequency encoding otherwise.\n\n"
    "Final Cleaned Dataset: (1000, ~40 features)."
)

# 5. Deliverables
doc.add_heading("5. Deliverables", level=1)
doc.add_paragraph(
    "- customer_churn_master.csv → raw integrated dataset.\n"
    "- eda_plots/ → visualizations.\n"
    "- eda_summaries/ → statistical summaries.\n"
    "- customer_churn_cleaned.csv → cleaned dataset ready for modeling.\n"
)

# 6. Next Steps
doc.add_heading("6. Next Steps", level=1)
doc.add_paragraph(
    "1. Model Training & Evaluation (Logistic Regression, Random Forest).\n"
    "2. Feature Importance (SHAP, permutation importance).\n"
    "3. Handle class imbalance (class weights, SMOTE).\n"
)

# Save Word file explicitly into project folder
save_path = r"C:\Projects\ChurnModel\Customer_Churn_Data_Preparation_Report.docx"
doc.save(save_path)

print(f"✅ Report saved at {save_path}")


✅ Report saved at C:\Projects\ChurnModel\Customer_Churn_Data_Preparation_Report.docx


PackageNotFoundError: Package not found at 'Customer_Churn_Data_Preparation_Report.docx'